In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
ci = pd.read_csv('ci_updated.csv')
rel = pd.read_csv('ci_rel_updated.csv').rename(columns={'chi_u_number' : 'CI number'})
pi = pd.read_csv('pi_updated.csv').rename(columns={'u_configuration_item.u_number' : 'CI number'})



In [ ]:
rels = rel.set_index('CI number').join(pi.set_index('CI number'))
rels = rels.reset_index(drop=True).rename(columns={'par_u_number' : 'CI number'}).set_index('CI number')
ci_child = rels[pd.notna(rels['u_product_item'])]['u_product_item']

ci = ci.rename(columns={'u_ci.u_number' : 'CI number'}).set_index('CI number')
ci = ci.join(pi.set_index('CI number'))

ci = ci.rename(columns={
    'u_ci' : 'Name',
    'u_ci.u_subclass' : 'Classification',
    'u_ci.company.u_top_company' : 'Company',
    'u_ci.model_id' : 'Model_ID',
    'u_ci.model_number' : 'Model_number',
    'u_ci.operational_status' : 'Operational_status',
    'u_product_item' : 'Product item',
    'u_calculation_type' : 'Calculation type',
    'u_ci.u_primary_bs'  : 'Business_service'
})

ci = ci.join(ci_child)
ci = ci.rename(columns={'u_product_item' : 'Child product item'})
dupl = pi[pi['CI number'].duplicated() == True]
dupl.rename(columns={'u_product_item' : 'Product item2'}, inplace=True)
ci = ci.join(dupl.set_index('CI number'))
tst = ci_child.to_frame()
tst= tst.reset_index()
duplPI2 = tst[tst['CI number'].duplicated() == True]
duplPI2.drop_duplicates(keep= 'first')
duplPI2.rename(columns= {'u_product_item' : 'Child Product Item2'} , inplace=True) 
ci_test = ci
ci_test = ci_test.join(duplPI2.set_index('CI number'))
ci_test = ci_test.drop_duplicates(keep='first', subset='Name')

ci_test = ci_test.drop(columns='u_calculation_type')

In [ ]:
ci_test.reset_index()

In [ ]:
ci_test['Business_service'].value_counts()

In [ ]:
itask_assigment = pd.read_csv('incident_task.csv')

In [ ]:
ci_test['ProductItem_Combined'] = " PI- " + ci_test['Product item'].astype(str) + " PI2- " + ci_test['Product item2'].astype(str) + " PI_CH- " + ci_test['Child product item'].astype(str) + " PI_CH2- " + ci_test['Child Product Item2'].astype(str)
ci_frame_combined = ci_test[[ 'Name', 'Classification', 'Company', 'Model_ID', 'Model_number', 'Operational_status', 'ProductItem_Combined', 'Business_service']]

ci_frame_combined

In [ ]:
ci_frame_combined= ci_frame_combined.reset_index()

In [ ]:
itask_assigment = itask_assigment.rename(columns={'cmdb_ci.u_number' : 'CI_number'})
ci_frame_combined = ci_frame_combined.rename(columns={'CI number' : 'CI_number'})

In [ ]:
# Create a new column in the first dataset with default value of False
ci_frame_combined['itask_created'] = False

# Iterate over each row in the first dataset and check if the "CI_number" is in the second dataset
for index, row in ci_frame_combined.iterrows():
    if row['CI_number'] in itask_assigment['CI_number'].values:
        # If the "CI_number" is in the second dataset, set the value of "itask_created" to True
        ci_frame_combined.at[index, 'itask_created'] = True


In [ ]:
ci_frame_combined